In [ ]:
import Pkg
Pkg.activate(".")

pkgs = [
"DataFrames",
"uCSV",
"ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
data_directory = joinpath(dirname(pwd()), "data")
results_dir =  joinpath(data_directory, "results")

In [ ]:
sample_directories = readdir(joinpath(data_directory, "SRA"), join=true)

In [ ]:
kraken_dbs = filter(x -> isdir(x), readdir("$(homedir())/workspace/kraken", join=true))

In [ ]:
# # adjust to database size
# kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_16gb_20231009"
# kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_08gb_20231009"
# kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_20221209"
kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_20231009"
memory = 256
threads = 32

In [ ]:
# use me to auto-configure sbatch parameterization
# host = first(readlines(`hostname`))
# @assert occursin(r"\.stanford\.edu", host)

In [ ]:
function read_kraken_report(kraken_report)
    kraken_report_header = [
        "percentage_of_fragments_at_or_below_taxon",
        "number_of_fragments_at_or_below_taxon",
        "number_of_fragments_assigned_directly_to_taxon",
        "rank",
        "ncbi_taxonid",
        "scientific_name"
    ]

    data, header = uCSV.read(kraken_report, delim='\t')
    kraken_report_table = DataFrames.DataFrame(data, kraken_report_header)
    kraken_report_table[!, "scientific_name"] = string.(strip.(kraken_report_table[!, "scientific_name"]))
    return kraken_report_table
end

In [ ]:
identifiers_to_rerun = [
"SRR6399724",
"SRR6399725",
"SRR6399726",
"SRR6399727",
"SRR6399728",
"SRR6399729",
"SRR6399730",
"SRR6399731",
"SRR6399732",
"SRR6399773",
"SRR6399810",
"SRR6399900",
"SRR6399901",
"SRR6399902",
"SRR6399903",
"SRR6399905",
"SRR6399906",
"SRR6399907",
"SRR6399908",
"SRR6399909",
"SRR6399910",
"SRR6399911",
"SRR6399912",
"SRR6399913",
"SRR6399914",
"SRR6399915",
"SRR6399916",
"SRR6399917",
"SRR6399918",
"SRR6399919",
"SRR6399920",
"SRR6399921",
"SRR6399922",
"SRR6399923",
"SRR6399924",
"SRR6399925",
"SRR6399926",
"SRR6399927",
"SRR6399928",
"SRR6399929",
"SRR6399930",
"SRR6399931",
"SRR6399932",
"SRR6399933",
"SRR6399934",
"SRR6399935",
"SRR6399936",
"SRR6399937",
"SRR6399938",
"SRR6399939",
"SRR6399940",
"SRR6399941",
"SRR6399942",
"SRR6399943",
"SRR6399944",
"SRR6399945",
"SRR6399946",
"SRR6399947",
"SRR6399948",
"SRR6399949",
"SRR6399950",
"SRR6399951",
"SRR6399952",
"SRR6399953",
"SRR6399954",
"SRR6399955",
"SRR6399956",
"SRR6399957",
"SRR6399958",
"SRR6399959",
"SRR6399960",
"SRR6399961",
"SRR6399962",
"SRR6399963",
"SRR6399964",
"SRR6399965",
"SRR6399966",
"SRR6399967",
"SRR6399968",
"SRR6399969",
"SRR6399970",
"SRR6399971",
"SRR6399972",
"SRR6399973",
"SRR6399974",
"SRR6399975",
"SRR6399976",
"SRR6399977",
"SRR6399978",
"SRR6399979",
"SRR6399980",
"SRR6399981",
"SRR6399982",
"SRR6399983",
"SRR6399984",
"SRR6399985",
"SRR6399986",
"SRR6399987",
"SRR6399988",
"SRR6399989",
"SRR6399990",
"SRR6399991",
"SRR6399992",
"SRR6399993",
"SRR6399994",
"SRR6399995",
"SRR6399996",
"SRR6399997",
"SRR6399998",
"SRR6399999",
"SRR6400000",
"SRR6400001",
"SRR6400002",
"SRR6400003",
"SRR6400004",
"SRR6400005",
"SRR6400006",
"SRR6400007",
"SRR6400008",
"SRR6400009",
"SRR6400010",
"SRR6400011",
"SRR6400012",
"SRR6400013",
"SRR6400014",
"SRR6400015",
"SRR6400016",
"SRR6400017",
"SRR6400018",
"SRR6400019",
"SRR6400021",
"SRR6400022",
"SRR6400023",
"SRR6400024",
"SRR7365458",
"SRR7365460",
"SRR7365461",
"SRR7365462",
"SRR7365463",
"SRR7365464",
"SRR7365465",
"SRR7365466",
"SRR7365467",
"SRR7365468",
"SRR7365469",
"SRR7365470",
"SRR7365471",
"SRR7365473",
"SRR7365474",
"SRR7365475",
"SRR7365476",
"SRR7365477",
"SRR7365478",
"SRR7365479",
"SRR7365480",
"SRR7365481",
"SRR7365482",
"SRR7365483",
"SRR7365484",
"SRR7365485",
]

In [ ]:
# kraken_results_dir = mkpath(joinpath(results_dir, "kraken"))
# # copy everything to results for quick transfer to NERSC
# ProgressMeter.@showprogress for sample_directory in sample_directories
#     sample = basename(sample_directory)
#     kraken_directory = joinpath(sample_directory, "kraken")
#     # kraken_outputs = filter(x -> occursin(r"kraken-output\.tsv$", x), readdir(kraken_directory, join=true))
#     # if !isempty(kraken_outputs)
#         # @assert length(kraken_outputs) == 1
#     kraken_reports = filter(x -> occursin(r"kraken-report\.tsv$", x), readdir(kraken_directory, join=true))
#     @assert length(kraken_reports) == 1
#     kraken_report = first(kraken_reports)
#     # @show kraken_report
#     copied_outfile = joinpath(kraken_results_dir, basename(kraken_report))
#     if !isfile(copied_outfile)
#         cp(kraken_report, copied_outfile)
#     end
#     # kraken_results_dir = mkpath(joinpath(results_dir, "kraken"))
#     # end
# end

In [ ]:
for sample_directory in sample_directories
    sample = basename(sample_directory)
    if basename(sample_directory) in Set(identifiers_to_rerun)
        kraken_directory = joinpath(sample_directory, "kraken")
        # display(readdir(kraken_directory))
        kraken_outputs = filter(x -> occursin(r"kraken-output\.tsv", x), readdir(kraken_directory, join=true))
        @assert isempty(kraken_outputs)
        # for bad_kraken_output in filter(x -> occursin("k2_pluspfp_20221209", x), readdir(kraken_directory, join=true))
        #     @show bad_kraken_output
        #     rm(bad_kraken_output)
        # end
        trimmed_forward_reads = joinpath(sample_directory, "trim_galore", "$(sample)_1_val_1.fq.gz")
        trimmed_reverse_reads = joinpath(sample_directory, "trim_galore", "$(sample)_2_val_2.fq.gz")
        
        # if isfile(trimmed_forward_reads) && isfile(trimmed_reverse_reads)
        # @show "ready to run"
        sample_id = basename(sample_directory)
        cmd = "julia $(homedir())/workspace/Mycelia/scripts/classify-short-reads.jl --forward_reads $(trimmed_forward_reads) --reverse_reads $(trimmed_reverse_reads) --kraken_db $(kraken_db) --threads $(threads) --out_directory $(kraken_directory)"
        submission = 
        `sbatch 
        --mail-user=cameron.prybol@gmail.com
        --mail-type=ALL
        --error=%x-%j-$(sample_id).err
        --output=%x-%j-$(sample_id).out
        --partition=batch
        --account=mpsnyder
        --nodes=1
        --ntasks=1
        --time=1-00:00:00    
        --cpus-per-task=$(threads)
        --mem=$(memory)G
        --job-name=classify-short-reads
        --wrap $cmd
        `
        
        run(submission)
        # else
        #     @show "waiting on" isfile(trimmed_forward_reads)
        #     @show "waiting on" isfile(trimmed_reverse_reads)
        # end

        
        # @show kraken_outputs
        # kraken_reports = filter(x -> occursin(r"kraken-report\.tsv$", x), readdir(kraken_directory, join=true))
        # @show kraken_reports
#         @assert length(kraken_reports) == 1
#         kraken_report = first(kraken_reports)
#         @show kraken_report
#         # @show filesize(kraken_report)
#         display(read_kraken_report(kraken_report)[1:2, :])
        
#         # confirm with trim_galore reports
#         trimming_reports = filter(x -> occursin(r"trimming_report.txt", x), readdir(joinpath(sample_directory, "trim_galore"), join=true))
#         display(trimming_reports)
#         display(readlines(first(trimming_reports)))
#         break
    end
    
    # matching_kraken_reports = filter(x -> occursin(basename(kraken_db), x), kraken_reports)
    # if isempty(matching_kraken_reports)
    #     # cmd = `julia $(homedir())/workspace/Mycelia/scripts/classify-short-reads.jl --forward_reads $(forward) --reverse_reads $(reverse) --kraken_db $(kraken_db) --threads $(threads)`
    #     # run(cmd)
    #     cmd = "julia $(homedir())/workspace/Mycelia/scripts/classify-short-reads.jl --forward_reads $(forward) --reverse_reads $(reverse) --kraken_db $(kraken_db) --threads $(threads)"
    #     submission = 
    #     `sbatch 
    #     --mail-user=cameron.prybol@gmail.com
    #     --mail-type=ALL
    #     --error=%x-%j.err
    #     --output=%x-%j.out
    #     --partition=batch
    #     --account=mpsnyder
    #     --nodes=1
    #     --ntasks=1
    #     --time=1-00:00:00    
    #     --cpus-per-task=$(threads)
    #     --mem=$(memory)G
    #     --job-name=classify-short-reads
    #     --wrap $cmd
    #     `
    #     run(submission)
    # end
end